In [115]:
import sqlite3
import pandas as pd
from binance.client import Client
from datetime import datetime
# Подключение к базе данных
conn = sqlite3.connect('trading_analysis.db')
df = pd.read_sql('select * from Trade_Signals where is_check = 0 or is_check = 3', con=conn)
df.drop(['analyseType', 'header', 'body'], axis=1, inplace=True)
df['createTime'] = pd.to_datetime(df['createTime'])
df['createTime'] = df['createTime'].dt.floor('T') - pd.to_timedelta(df['createTime'].dt.minute % 5, unit='m')
df['foreCast'] = df['foreCast'].replace({'Up' : 'Buy', 'Down': 'Sell'})
df.head()

,id,instrumentId,createTime,foreCast,entry,tp1,tp2,sl,is_check,status,time_enter,time_close,price_close,duration
0,22730993,SOLUSDT,2024-12-15 13:30:00,Buy,220.3700,226.6100,229.73000,215.56000,0,,,,,
1,22736543,BTCUSDT,2024-12-15 14:25:00,Sell,102926.3000,100561.3000,NaN,103753.13000,0,,,,,
2,22736743,ETHUSDT,2024-12-15 15:00:00,Sell,3905.8300,3829.5000,3783.94000,3946.46000,0,,,,,
3,22736893,SOLUSDT,2024-12-15 17:00:00,Sell,222.0200,215.1000,209.72000,226.12000,0,,,,,
4,22737243,ADAUSDT,2024-12-15 18:00:00,Sell,1.0874,1.0384,0.94505,1.13485,0,,,,,


In [56]:
def get_hist_data(symbol: str, start_time: str) -> pd.DataFrame:
    api_key = 'd1bc948a8af0c74cc5c746f68e5b63bf21ab8789533b637eb6f80d754508b66a'
    api_secret = '81929ddcab087592ac2ce85d1a6933b9379f2fcf458077b85fb7d0666ab23b13'
    client = Client(api_key, api_secret)
    interval = Client.KLINE_INTERVAL_5MINUTE 
    end_time = None
    candles = client.get_historical_klines(symbol, interval, start_time, end_time)
    df = pd.DataFrame(candles, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                        'Close Time', 'Quote Asset Volume', 'Number of Trades',
                                        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df.drop(['Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'], axis=1, inplace=True)
    df[['Open', 'High', "Low", 'Close']] = df[['Open', 'High', "Low", 'Close']].apply(pd.to_numeric, errors='coerce')
    return df

symbol = 'DOGEUSDT'  # Торговая пара
start_time = '2024-12-10 21:01:00'
data = get_hist_data(symbol, start_time).head()

In [9]:
df[df['instrumentId'] == 'ETHUSDT']['createTime'].min()

Timestamp('2024-12-10 12:30:00')

In [66]:
data.head()

,Open time,Open,High,Low,Close
0,2024-12-12 11:00:00,0.41169,0.41207,0.40955,0.40957
1,2024-12-12 11:05:00,0.40957,0.41187,0.40956,0.41041
2,2024-12-12 11:10:00,0.41040,0.41040,0.40858,0.40922
3,2024-12-12 11:15:00,0.40923,0.41126,0.40922,0.41030
4,2024-12-12 11:20:00,0.41030,0.41064,0.40900,0.41003


In [109]:

def Strategy_Tester_v3(start_time: pd.Timestamp, df: pd.DataFrame, side: str, price: float, stop_loss: float, take_profit1: float, take_profit2: float = None, open_order_type: str = 'Limit') -> tuple:
    df = df[df['Open time'] >= start_time].reset_index(drop=True)
    
    side, open_order_type = side.capitalize(), open_order_type.capitalize()
    PRICE_INDICATOR = False
    STOP_LOSS_INDICATOR = False
    TAKE_PROFIT1_INDICATOR = False
    TAKE_PROFIT2_INDICATOR = False
    approximate_time_TP1, approximate_time_TP2, approximate_time_SL = None, None, None


    # Флаг направления сделки
    flag_buy = side == 'Buy'

    approximate_time_enter = None

    for i, row in df.iterrows():

        if row['Open time'] >= start_time + pd.Timedelta(hours=24):
            return ["NOT_ENTER"] 

        # Проверяем вход по цене
        if not PRICE_INDICATOR:
            if (flag_buy and row['Low'] <= price) or (not flag_buy and row['High'] >= price):
                PRICE_INDICATOR = True
                approximate_time_enter = row['Open time']
            else:
                continue  # Пока не вошли в позицию, пропускаем строку

        # Проверяем SL, TP1, TP2
        if PRICE_INDICATOR:
            # Проверка Stop Loss
            if (flag_buy and row['Low'] <= stop_loss) or (not flag_buy and row['High'] >= stop_loss):
                STOP_LOSS_INDICATOR = True
                approximate_time_SL = row['Open time']

                if TAKE_PROFIT1_INDICATOR: # если вдруг TP1 отработал, а потом сработал стоп лосс
                    return ("TP1", approximate_time_enter, approximate_time_TP1, (approximate_time_TP1 - approximate_time_TP1), take_profit1)

                return ("SL", approximate_time_enter, approximate_time_SL, (approximate_time_SL - approximate_time_enter), stop_loss)

            # Проверка Take Profit 1
            if not TAKE_PROFIT1_INDICATOR:
                if (flag_buy and row['High'] >= take_profit1) or (not flag_buy and row['Low'] <= take_profit1):
                    TAKE_PROFIT1_INDICATOR = True
                    approximate_time_TP1 = row['Open time']

            if take_profit2:
            # Проверка Take Profit 2
                if not TAKE_PROFIT2_INDICATOR:
                    if (flag_buy and row['High'] >= take_profit2) or (not flag_buy and row['Low'] <= take_profit2):
                        TAKE_PROFIT2_INDICATOR = True
                        approximate_time_TP2 = row['Open time']
                        return ("TP2", approximate_time_enter, approximate_time_TP2, (approximate_time_TP2 - approximate_time_enter), take_profit2)

            # Если достигнут только TP1
            if TAKE_PROFIT1_INDICATOR and not TAKE_PROFIT2_INDICATOR:
                return ("TP1",  approximate_time_enter, approximate_time_TP1, (approximate_time_TP1 - approximate_time_enter), take_profit1)

    # Если не было выполнено ни одно из условий (SL, TP1, TP2), возвращаем EXP
    if PRICE_INDICATOR:
        last_price = row['Close']
        approximate_time_EXP = row['Open time']
        return ("EXP", approximate_time_enter, approximate_time_EXP, (approximate_time_EXP - approximate_time_enter), last_price)

    return ["NOT_ENTER"] 

def timedelta(str1, str2):
    date_format = '%Y-%m-%d %H:%M:%S'
    date1 = datetime.strptime(str1, date_format)
    date2 = datetime.strptime(str2, date_format)
    return str(date2 - date1)



In [110]:
import numpy as np
import time

# np.isnan(value):


In [111]:
# Исходный кортеж
my_tuple = (2, 3, 4)

# Добавляем значение на первое место
new_value = 1
updated_tuple = (new_value,) + my_tuple

print(updated_tuple)  # (1, 2, 3, 4)


(1, 2, 3, 4)


In [112]:
res = []
for symbol in df['instrumentId'].unique():
   min_time = df[df['instrumentId'] == symbol]['createTime'].min()
   data = get_hist_data(symbol=symbol, start_time=str(min_time))

   for index, row in df[df['instrumentId'] == symbol].iterrows():
      enter_price = row['entry']
      tp1 = row['tp1']
      tp2 = row['tp2']
      sl = row['sl']
      if not np.isnan(enter_price) and not np.isnan(tp1) and not np.isnan(sl):
         if np.isnan(tp2):
            tp2 = None
         result = list(Strategy_Tester_v3(
            start_time=row['createTime'],
            df=data,
            side=row['foreCast'],
            price=enter_price,
            stop_loss=sl,
            take_profit1=tp1,
            take_profit2=tp2
         ))
         # Добавляем `id` к результату
         result.insert(0, row['id'])
         res.append(result)
      else:
         res.append([row['id'], 'incorrect data'])
         


   
    

In [113]:
import sqlite3

# Подключаемся к базе данных
conn = sqlite3.connect('trading_analysis.db')
cursor = conn.cursor()
sql1 = 'UPDATE Trade_Signals SET status = ?, time_enter = ?, time_close = ?, duration = ?, price_close = ?, is_check = 1 WHERE id = ?'
sql2 = 'UPDATE Trade_Signals SET status = ?, is_check = 1  WHERE id = ?'
for values in res:
    if len(values) == 6:
        cursor.execute(sql1, (values[1], str(values[2]), str(values[3]), str(values[4]), values[5], values[0]))
    else:
        cursor.execute(sql2, (values[1],  values[0]))

# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

In [100]:
res1 = []
res1.append([row['id'], 'incorrect data', 'No data', 'No data', 'No data', 'No data'])
res1

[[22535093, 'incorrect data', 'No data', 'No data', 'No data', 'No data']]

In [106]:
res1[0][2]

'No data'